In [5]:
import os
import numpy as np
import torchvision.models as models
from PIL import Image
import matplotlib.pyplot as plt


import torch
import torchvision
import torchinfo
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.io import read_image
from torchvision.transforms import functional as TVF
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split
from tqdm import tqdm
from torchinfo import summary


data_dir = "MY_data"
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [6]:
train_transform = transforms.Compose([
    transforms.Resize((40, 40)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(40),                  
    transforms.ToTensor(),
])

trainset = ImageFolder(data_dir + "/train", transform=train_transform)

train_size = int(0.8 * len(trainset)) 
val_size = len(trainset) - train_size 

trainSubset, valSubset = random_split(trainset, [train_size, val_size])

train_loader = DataLoader(trainSubset, batch_size=64, shuffle=True)
val_loader = DataLoader(valSubset, batch_size=64, shuffle=False)


In [7]:
#trainset = ImageFolder(data_dir + "/train", transform=train_transform)
#train_size = int(0.8 * len(trainset)) 
#val_size = len(trainset) - train_size 

#trainSubset, valSubset = random_split(trainset, [train_size, val_size])

#train_loader = DataLoader(trainSubset, batch_size =64, shuffle=True)
#val_loader = DataLoader(valSubset, batch_size=64, shuffle=False)

In [8]:
import matplotlib.pyplot as plt

def plot_training_results(train_losses, val_losses, train_accuracies, val_accuracies):
    
    epochs = range(1, len(train_losses) + 1)

    # Plotting loss
    plt.figure(figsize=(14, 6))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label='Training Loss')
    plt.plot(epochs, val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()

    # Plotting accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label='Training Accuracy')
    plt.plot(epochs, val_accuracies, label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()

    plt.show()


In [9]:
def train(net, trainloader, valloader, device, num_epochs, lr=0.01, momentum=0.8, step_size=5, gamma=0.1, verbose=True):
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []

    loss_iterations = int(np.ceil(len(trainloader) / 3))
    
    # Transfer model to GPU
    net = net.to(device)

    # Set the optimizer using the lr and momentum settings passed by the user
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

    for epoch in range(num_epochs):
        running_loss = 0
        running_corrects = 0
        total_samples = 0

        # Training phase
        net.train()  # Ensure the model is in training mode
        for i, (inputs, labels) in enumerate(trainloader):
            optimizer.zero_grad()

            # Transfer data to GPU
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward propagation to get outputs
            outputs = net(inputs)

            # Compute loss
            loss = F.cross_entropy(outputs, labels)

            # Backpropagation to get gradients of all parameters
            loss.backward()

            # Update parameters
            optimizer.step()

            # Accumulate loss
            running_loss += loss.item() * inputs.size(0)

            # Calculate accuracy for the batch
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels)
            total_samples += inputs.size(0)

        # Calculate average training loss and accuracy
        train_loss = running_loss / total_samples
        train_acc = running_corrects.float() / total_samples
        train_losses.append(train_loss)
        train_accuracies.append(train_acc.cpu().numpy())

        # Validation phase
        net.eval()  # Set model to evaluation mode
        val_loss = 0
        val_corrects = 0
        val_samples = 0

        with torch.no_grad():  # Disable gradient computation
            for inputs, labels in valloader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = net(inputs)
                loss = F.cross_entropy(outputs, labels)

                val_loss += loss.item() * inputs.size(0)

                _, preds = torch.max(outputs, 1)
                val_corrects += torch.sum(preds == labels)
                val_samples += inputs.size(0)

        val_loss /= val_samples
        val_acc = val_corrects.double() / val_samples
        val_losses.append(val_loss)
        val_accuracies.append(val_acc.cpu().numpy())

        print(f'[Epoch {epoch+1:2d}]: train_loss = {train_loss:.4f}, train_acc = {train_acc:.4f}, '
              f'validation_loss = {val_loss:.4f}, validation_acc = {val_acc:.4f}')

        scheduler.step()  # Step the learning rate scheduler

    print("Training completed.")
    return train_losses, val_losses, train_accuracies, val_accuracies

In [10]:
import torch
from sklearn.metrics import precision_score, recall_score, f1_score

def evaluate(net, dataloader, device):
    net.eval()  # Set model to evaluation mode
    all_targets = []
    all_predictions = []

    with torch.no_grad():  # Disable gradient computation
        for inputs, targets in dataloader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = net(inputs)
            _, predicted = torch.max(outputs, 1)  # Extract the predicted class indices
            
            all_targets.extend(targets.cpu().numpy())  # Store the true labels
            all_predictions.extend(predicted.cpu().numpy())  # Store the predicted labels

    # Calculate accuracy
    accuracy = sum(np.array(all_predictions) == np.array(all_targets)) / len(all_targets)
    print(f'Accuracy: {accuracy:.4f}')
    
    # Calculate Precision, Recall, and F1-Score
    precision = precision_score(all_targets, all_predictions, average='weighted')
    recall = recall_score(all_targets, all_predictions, average='weighted')
    f1 = f1_score(all_targets, all_predictions, average='weighted')

    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1-Score: {f1:.4f}')

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomModel(nn.Module):
    def __init__(self, num_classes=34):
        super(CustomModel, self).__init__()

        # Convolutional Layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=0)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, padding=2)
        self.conv2_1 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, padding=2)
        self.batchnorm1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=5, padding=2)
        self.conv4_1 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=5, padding=2)
        self.conv3_1 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.4)
        
        self.conv5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv5_1 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=5, padding=2)
        self.conv5_2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=5, padding=2)
        self.conv5_3 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=5, padding=2)
        self.conv5_4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=5, padding=2)
        self.conv5_5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.conv5_6 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.batchnorm3 = nn.BatchNorm2d(128)
        self.dropout2 = nn.Dropout(0.6)

        self.fc1 = nn.Linear(128, 64)  
        self.batchnorm4 = nn.BatchNorm1d(64)
        self.dropout3 = nn.Dropout(0.7)
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        # Forward pass
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv2_1(x))
        x = self.batchnorm1(x)
        x = self.pool1(x)
        
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv4_1(x))
        x = F.relu(self.conv3_1(x))
        
        x = self.pool2(x)
        x = self.batchnorm2(x)
        x = self.dropout1(x)
        
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv5_6(x))
        x = self.avgpool(x)
        
        x = self.batchnorm3(x)

        x = torch.flatten(x, 1)
    
        x = F.relu(self.fc1(x))
        x = self.batchnorm4(x)
        x = self.dropout3(x)
        x = self.fc2(x)
    
        return x

num_classes = 10 
net = CustomModel(num_classes=num_classes)

from torchinfo import summary
summary(net, input_size=(64, 3, 40, 40))

Layer (type:depth-idx)                   Output Shape              Param #
CustomModel                              [64, 10]                  1,934,080
├─Conv2d: 1-1                            [64, 16, 38, 38]          448
├─Conv2d: 1-2                            [64, 32, 38, 38]          12,832
├─Conv2d: 1-3                            [64, 32, 38, 38]          25,632
├─BatchNorm2d: 1-4                       [64, 32, 38, 38]          64
├─MaxPool2d: 1-5                         [64, 32, 19, 19]          --
├─Conv2d: 1-6                            [64, 64, 19, 19]          18,496
├─Conv2d: 1-7                            [64, 64, 19, 19]          102,464
├─Conv2d: 1-8                            [64, 64, 19, 19]          102,464
├─Conv2d: 1-9                            [64, 64, 19, 19]          36,928
├─MaxPool2d: 1-10                        [64, 64, 9, 9]            --
├─BatchNorm2d: 1-11                      [64, 64, 9, 9]            128
├─Dropout: 1-12                          [64, 64, 

In [12]:
# class NeuralNetwork(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=13, kernel_size=5, padding=1)
#         self.act1 = nn.ReLU()
#         self.pool1 = nn.AvgPool2d(kernel_size=3, stride=2)
        
#         self.conv2 = nn.Conv2d(in_channels=13, out_channels=27, kernel_size=5, padding=1)
#         self.act2 = nn.ReLU()
#         self.pool2 = nn.AvgPool2d(kernel_size=3, stride=2)
        
#         #6250
#         self.fc1 = nn.Linear(1323, 3100)

#         self.act3 = nn.ReLU()
        
#         self.fc2 = nn.Linear(3100, 400)
#         self.act4 = nn.Sigmoid()
        
#         self.fc3 = nn.Linear(400, 10)
        
        
#     def forward(self, x):
        
#         x = self.conv1(x)
#         x = self.act1(x)
#         x = self.pool1(x)
        
#         x = self.conv2(x)
#         x = self.act2(x)
#         x = self.pool2(x)  
        
        
#         x = x.view(x.size(0), x.size(1) * x.size(2) * x.size(3))
#         x = self.fc1(x)
#         x = self.act3(x)
        
#         x = self.fc2(x)
#         x = self.act4(x)
        
#         x = self.fc3(x)
#         return x

# net = NeuralNetwork()
# summary(net, input_size=(64, 3, 40, 40))


In [13]:
train_losses, val_losses, train_accuracies, val_accuracies = train(net, train_loader, val_loader, device, num_epochs=30, lr=0.01, step_size=5, gamma=0.1)

[Epoch  1]: train_loss = 2.4925, train_acc = 0.1370, validation_loss = 2.1812, validation_acc = 0.1866
[Epoch  2]: train_loss = 2.1779, train_acc = 0.2043, validation_loss = 2.0523, validation_acc = 0.2343
[Epoch  3]: train_loss = 2.1076, train_acc = 0.2429, validation_loss = 2.2091, validation_acc = 0.2061
[Epoch  4]: train_loss = 2.0665, train_acc = 0.2380, validation_loss = 2.2449, validation_acc = 0.1735
[Epoch  5]: train_loss = 2.0151, train_acc = 0.2511, validation_loss = 2.0673, validation_acc = 0.2148
[Epoch  6]: train_loss = 1.9536, train_acc = 0.2755, validation_loss = 1.9254, validation_acc = 0.2495
[Epoch  7]: train_loss = 1.9422, train_acc = 0.2815, validation_loss = 1.9122, validation_acc = 0.2690
[Epoch  8]: train_loss = 1.9122, train_acc = 0.3022, validation_loss = 1.9176, validation_acc = 0.2798
[Epoch  9]: train_loss = 1.9143, train_acc = 0.3043, validation_loss = 1.8855, validation_acc = 0.3210
[Epoch 10]: train_loss = 1.9114, train_acc = 0.2962, validation_loss = 1.

KeyboardInterrupt: 

In [14]:
plot_training_results(train_losses, val_losses, train_accuracies, val_accuracies)

NameError: name 'train_losses' is not defined

In [16]:
test_transform = transforms.Compose([
    transforms.Resize((40, 40)),
    transforms.ToTensor(),
])

testset = ImageFolder(data_dir + "/Test", transform=test_transform)
test_loader = DataLoader(testset, batch_size=64, shuffle=False)
evaluate(net, test_loader, device)

Accuracy: 0.2615
Precision: 0.2804
Recall: 0.2615
F1-Score: 0.2013


C:\Users\GF63\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
def predict_image(image_path, model, transform=None):
    model.eval()
    
    device = next(model.parameters()).device
    
    image = Image.open(image_path)
    
    img = transform(image)
    
    img = img.to(device)
    
    xb = img.unsqueeze(0)
    
    with torch.no_grad():
        yb = model(xb)
        
    _, preds = torch.max(yb, dim=1)
    
    return preds.item()